## 2: AgentCore Identity

This notebook configures secure authentication for AgentCore Runtime & AgentCore Gateway using Amazon Cognito and JWT tokens. You'll set up user pools, create credentials, and generate bearer tokens for authenticated agent invocations.

**Prerequisites:** Completed 1, AWS permissions for Cognito

### Import Required Libraries

In [ ]:
import boto3
from utils.identity_ssm_utils import setup_cognito_user_pool, reauthenticate_user

# Get AWS session information
session = boto3.Session()
region = session.region_name or 'us-west-2'

sts = session.client('sts')
identity = sts.get_caller_identity()
account_id = identity['Account']

print(f"Account ID: {account_id}")
print(f"Region: {region}")

### Setting Up Amazon Cognito User Pool

Amazon Cognito provides user authentication for AgentCore Runtime and AgentCore Gateway. This creates a User Pool, App Client, and test user.

**Note:** Access tokens are valid for 2 hours. Use `reauthenticate_user()` to generate new tokens.

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")

# Display important configuration details
print("\n" + "=" * 80)
print("Cognito Configuration:")
print("=" * 80)
print(f"Pool ID: {cognito_config.get('pool_id')}")
print(f"Client ID: {cognito_config.get('client_id')}")
print(f"Discovery URL: {cognito_config.get('discovery_url')}")
print(f"\nBearer Token (valid for 2 hours):")
print(f"{cognito_config.get('bearer_token')[:50]}...")
print("=" * 80)

### Creating IAM Execution Role

AgentCore Runtime needs an IAM role with permissions to access Bedrock models, Knowledge Bases, Memory, and CloudWatch logs.

In [ ]:
from utils.identity_ssm_utils import create_agentcore_runtime_execution_role

# Create execution role for AgentCore Runtime
agentcore_runtime_execution_role = create_agentcore_runtime_execution_role()

print(f"\nExecution Role ARN: {agentcore_runtime_execution_role}")

### Refreshing Authentication Tokens

Cognito tokens expire after 2 hours. Use this cell to generate a fresh token:

In [ ]:
# Refresh the bearer token
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"),
    cognito_config.get("client_secret")
)

print("✅ New bearer token generated!")
print(f"Token (first 50 chars): {bearer_token[:50]}...")
print("\n⏰ This token is valid for 2 hours from now.")

### Saving Configuration for Later Use

Store the Cognito configuration for use in other notebooks:

In [ ]:
import json

# Save configuration to a file for easy access
config_data = {
    "pool_id": cognito_config.get("pool_id"),
    "client_id": cognito_config.get("client_id"),
    "client_secret": cognito_config.get("client_secret"),
    "discovery_url": cognito_config.get("discovery_url"),
    "execution_role": agentcore_runtime_execution_role
}

with open('cognito_config.json', 'w') as f:
    json.dump(config_data, f, indent=2)

print("✅ Configuration saved to cognito_config.json")

### Summary

You've configured Amazon Cognito for authentication, created IAM execution roles, and learned how to generate and refresh bearer tokens for secure agent invocations.

### Next Steps

- **3: AgentCore Gateway** - Create gateway for tool exposure